In [1]:
# import os
# import pandas as pd
# for dirname, _, filenames in os.walk('../Datasets/Pistachio_Image_Dataset/'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))


# dfMerge = pd.read_excel('../Datasets/Pistachio_Image_Dataset/Pistachio_28_Features_Dataset/Pistachio_28_Features_Dataset.xlsx')

In [2]:
import tensorflow as tf

mnist = tf.keras.datasets.mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train, X_test = X_train / 255.0, X_test / 255.0

2023-03-09 21:01:15.680659: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-09 21:01:17.905307: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib/cuda/include:/usr/lib/cuda/lib64:
2023-03-09 21:01:17.906054: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib/cuda/include:/usr/lib/cuda/lib64:
2023-03-09 21:01:17.906071: W tensorflow/com

In [3]:
from tensorflow.keras.callbacks import TensorBoard
log_folder = 'logs'

In [4]:
callbacks = [TensorBoard(log_dir=log_folder,
                         histogram_freq=1,
                         write_graph=True,
                         write_images=True,
                         update_freq='epoch',
                         profile_batch=1,
                         embeddings_freq=1)]

2023-03-09 21:01:21.411244: I tensorflow/core/profiler/lib/profiler_session.cc:101] Profiler session initializing.
2023-03-09 21:01:21.411333: I tensorflow/core/profiler/lib/profiler_session.cc:116] Profiler session started.
2023-03-09 21:01:23.279040: I tensorflow/core/profiler/backends/gpu/cupti_tracer.cc:1664] Profiler found 1 GPUs
2023-03-09 21:01:23.281723: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcupti.so.11.2'; dlerror: libcupti.so.11.2: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib/cuda/include:/usr/lib/cuda/lib64:
2023-03-09 21:01:23.321950: I tensorflow/core/profiler/lib/profiler_session.cc:128] Profiler session tear down.
2023-03-09 21:01:23.322363: I tensorflow/core/profiler/backends/gpu/cupti_tracer.cc:1798] CUPTI activity buffer flushed


In [5]:
# try:
#    model =  tf.keras.models.load_model('tf/mnist')
#    print('Loading Model')
# except:
print('Creating Model')
model = tf.keras.models.Sequential([
   tf.keras.layers.Flatten(input_shape=(28, 28)),
   tf.keras.layers.Dense(512, activation='relu'),
   tf.keras.layers.Dropout(0.2),
   tf.keras.layers.Dense(10, activation='softmax')])

model.compile(optimizer='sgd',
   loss='sparse_categorical_crossentropy',
   metrics=['accuracy'])
model.fit(X_train, y_train,
         epochs=10,
         validation_split=0.2,
         callbacks=callbacks)
model.save('tf/mnist')

2023-03-09 21:01:23.406949: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


Creating Model


2023-03-09 21:01:23.599354: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-03-09 21:01:23.599688: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-03-09 21:01:23.603415: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-09 21:01:23.615064: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_nod

Epoch 1/10


2023-03-09 21:01:27.433852: I tensorflow/core/profiler/lib/profiler_session.cc:101] Profiler session initializing.
2023-03-09 21:01:27.433914: I tensorflow/core/profiler/lib/profiler_session.cc:116] Profiler session started.
2023-03-09 21:01:32.580620: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x7f20ef85a6f0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-03-09 21:01:32.586568: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): NVIDIA GeForce GTX 1650, Compute Capability 7.5
2023-03-09 21:01:33.823174: W tensorflow/compiler/xla/service/gpu/nvptx_helper.cc:56] Can't find libdevice directory ${CUDA_DIR}/nvvm/libdevice. This may result in compilation or runtime failures, if the program we try to run uses routines from libdevice.
Searched for CUDA in the following directories:
  ./cuda_sdk_lib
  /usr/local/cuda-11.2
  /usr/local/cuda
  .
You can choose the search directory by setting xla_gpu_cuda_d

: 

: 

In [ ]:
%load_ext tensorboard
log_folder = 'logs'

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [ ]:
%tensorboard --logdir={log_folder}